In [13]:
"""
Enhanced Parametric Spectral Network for Human Activity Recognition
- 성능 개선: Multi-resolution freq init, Complex weights, Gating fusion
- Data augmentation 없이 architectural improvements만 적용
"""

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import numpy as np
import os
from sklearn.metrics import accuracy_score, f1_score
import math

# ============================================================================
# 1. Data Loading (UCI-HAR)
# ============================================================================

class UCIHARDataset(Dataset):
    def __init__(self, data_dir, split='train'):
        self.split = split

        if split == 'train':
            signals_dir = os.path.join(data_dir, 'train', 'Inertial Signals')
            labels_file = os.path.join(data_dir, 'train', 'y_train.txt')
        else:
            signals_dir = os.path.join(data_dir, 'test', 'Inertial Signals')
            labels_file = os.path.join(data_dir, 'test', 'y_test.txt')

        signal_types = [
            'body_acc_x', 'body_acc_y', 'body_acc_z',
            'body_gyro_x', 'body_gyro_y', 'body_gyro_z',
            'total_acc_x', 'total_acc_y', 'total_acc_z'
        ]

        signals = []
        for signal_type in signal_types:
            filename = f"{signal_type}_{split}.txt"
            filepath = os.path.join(signals_dir, filename)
            signal = np.loadtxt(filepath)
            signals.append(signal)

        self.data = np.stack(signals, axis=1).astype(np.float32)
        self.labels = np.loadtxt(labels_file, dtype=np.int64) - 1

        print(f"[{split}] Loaded {len(self.labels)} samples, shape: {self.data.shape}")

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return torch.from_numpy(self.data[idx]), self.labels[idx]

# ============================================================================
# 2. Enhanced Parametric Spectral Operator
# ============================================================================

class EnhancedSpectralOperator(nn.Module):
    """
    개선사항:
    1. Multi-resolution frequency initialization (low/mid/high bands)
    2. Complex-valued channel weights (real + imaginary)
    3. Channel-wise learnable normalization
    """
    def __init__(self, seq_len, in_channels, num_basis=64):
        super().__init__()
        self.seq_len = seq_len
        self.in_channels = in_channels
        self.num_basis = num_basis

        self.register_buffer('t', torch.arange(seq_len).float())

        # Multi-resolution initialization
        low_band = num_basis // 3
        mid_band = num_basis // 3
        high_band = num_basis - low_band - mid_band

        freqs_low = torch.linspace(0, seq_len//8, low_band) / seq_len
        freqs_mid = torch.linspace(seq_len//8, seq_len//4, mid_band) / seq_len
        freqs_high = torch.linspace(seq_len//4, seq_len//2, high_band) / seq_len
        init_freqs = torch.cat([freqs_low, freqs_mid, freqs_high])

        self.frequencies = nn.Parameter(init_freqs)
        self.phases = nn.Parameter(torch.zeros(num_basis))

        # Complex-valued weights
        self.channel_weights_real = nn.Parameter(torch.randn(in_channels, num_basis) * 0.02)
        self.channel_weights_imag = nn.Parameter(torch.randn(in_channels, num_basis) * 0.02)

        # Channel-wise normalization
        self.gamma = nn.Parameter(torch.ones(in_channels, 1))
        self.beta = nn.Parameter(torch.zeros(in_channels, 1))

    def forward(self, x):
        B, C, T = x.shape

        # Construct basis
        args = 2 * np.pi * self.frequencies.unsqueeze(1) * self.t.unsqueeze(0) + self.phases.unsqueeze(1)
        basis_real = torch.cos(args)
        basis_imag = -torch.sin(args)

        # Spectral transform
        spectral_real = torch.einsum('bct,ft->bcf', x, basis_real)
        spectral_imag = torch.einsum('bct,ft->bcf', x, basis_imag)

        # Complex multiplication: (a + bi) * (c + di) = (ac - bd) + (ad + bc)i
        weighted_real = spectral_real * (1 + self.channel_weights_real.unsqueeze(0)) - \
                       spectral_imag * self.channel_weights_imag.unsqueeze(0)
        weighted_imag = spectral_real * self.channel_weights_imag.unsqueeze(0) + \
                       spectral_imag * (1 + self.channel_weights_real.unsqueeze(0))

        # Magnitude
        spectral_mag = torch.sqrt(weighted_real**2 + weighted_imag**2 + 1e-8)
        spectral_mag = spectral_mag / math.sqrt(self.seq_len)

        # Channel-wise normalization
        spectral_mag = spectral_mag * self.gamma.unsqueeze(0) + self.beta.unsqueeze(0)

        return spectral_mag

# ============================================================================
# 3. Enhanced Attention Modules
# ============================================================================

class SpectralAttention(nn.Module):
    def __init__(self, num_basis, num_heads=4, dropout=0.1):
        super().__init__()
        self.num_heads = num_heads
        self.head_dim = num_basis // num_heads

        self.qkv = nn.Linear(num_basis, num_basis * 3)
        self.proj = nn.Linear(num_basis, num_basis)
        self.dropout = nn.Dropout(dropout)
        self.norm = nn.LayerNorm(num_basis)

        # Learnable temperature
        self.temperature = nn.Parameter(torch.ones(1) * math.sqrt(self.head_dim))

    def forward(self, x):
        B, C, F_dim = x.shape

        outputs = []
        for c in range(C):
            x_c = x[:, c, :]
            x_norm = self.norm(x_c)

            qkv = self.qkv(x_norm).reshape(B, 3, self.num_heads, self.head_dim).permute(1, 0, 2, 3)
            q, k, v = qkv[0], qkv[1], qkv[2]

            attn = (q @ k.transpose(-2, -1)) / self.temperature
            attn = F.softmax(attn, dim=-1)
            attn = self.dropout(attn)

            out = (attn @ v).reshape(B, F_dim)
            out = self.proj(out)
            out = self.dropout(out)
            out = out + x_c
            outputs.append(out)

        return torch.stack(outputs, dim=1)

class EnhancedCrossDomainFusion(nn.Module):
    """
    개선사항:
    1. Bidirectional cross-attention
    2. Adaptive gating mechanism
    3. Feed-forward network
    """
    def __init__(self, hidden_dim, num_heads=4, dropout=0.1):
        super().__init__()
        self.cross_attn1 = nn.MultiheadAttention(hidden_dim, num_heads, dropout=dropout, batch_first=True)
        self.cross_attn2 = nn.MultiheadAttention(hidden_dim, num_heads, dropout=dropout, batch_first=True)

        # Gating mechanism
        self.gate = nn.Sequential(
            nn.Linear(hidden_dim * 2, hidden_dim),
            nn.Sigmoid()
        )

        # FFN
        self.ffn = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim * 4),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim * 4, hidden_dim),
            nn.Dropout(dropout)
        )

        self.norm1 = nn.LayerNorm(hidden_dim)
        self.norm2 = nn.LayerNorm(hidden_dim)
        self.norm3 = nn.LayerNorm(hidden_dim)
        self.norm4 = nn.LayerNorm(hidden_dim)

        self.dropout = nn.Dropout(dropout)

    def forward(self, h_temporal, h_spectral):
        # Cross attention
        attn_ts, _ = self.cross_attn1(self.norm1(h_temporal), self.norm2(h_spectral), h_spectral)
        attn_st, _ = self.cross_attn2(self.norm3(h_spectral), self.norm4(h_temporal), h_temporal)

        h_t = h_temporal + self.dropout(attn_ts)
        h_s = h_spectral + self.dropout(attn_st)

        # Pooling
        pooled_t = h_t.mean(dim=1)  # (B, D)
        pooled_s = h_s.mean(dim=1)  # (B, D)

        # Adaptive gating
        gate_input = torch.cat([pooled_t, pooled_s], dim=-1)
        gate_weight = self.gate(gate_input)

        # Gated fusion
        fused = gate_weight * pooled_t + (1 - gate_weight) * pooled_s

        # FFN
        fused = fused + self.ffn(self.norm4(fused))

        return fused

# ============================================================================
# 4. Enhanced Temporal Encoder
# ============================================================================

class EnhancedTemporalEncoder(nn.Module):
    def __init__(self, in_channels, hidden_dim, num_heads=4, dropout=0.1):
        super().__init__()
        # Multi-scale convolution
        self.conv1 = nn.Conv1d(in_channels, hidden_dim, 3, padding=1)
        self.conv2 = nn.Conv1d(in_channels, hidden_dim, 5, padding=2)
        self.conv3 = nn.Conv1d(in_channels, hidden_dim, 7, padding=3)

        self.fusion_conv = nn.Conv1d(hidden_dim * 3, hidden_dim, 1)

        self.attn = nn.MultiheadAttention(hidden_dim, num_heads, dropout=dropout, batch_first=True)
        self.norm1 = nn.LayerNorm(hidden_dim)
        self.norm2 = nn.LayerNorm(hidden_dim)

        self.ffn = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim * 4),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim * 4, hidden_dim),
            nn.Dropout(dropout)
        )

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # Multi-scale feature extraction
        h1 = F.relu(self.conv1(x))
        h2 = F.relu(self.conv2(x))
        h3 = F.relu(self.conv3(x))

        h = torch.cat([h1, h2, h3], dim=1)
        h = self.fusion_conv(h)  # (B, D, T)

        h = h.transpose(1, 2)  # (B, T, D)

        # Self-attention
        h_norm = self.norm1(h)
        attn_out, _ = self.attn(h_norm, h_norm, h_norm)
        h = h + self.dropout(attn_out)

        # FFN
        h = h + self.ffn(self.norm2(h))

        return h

# ============================================================================
# 5. Full Enhanced Model
# ============================================================================

class EnhancedParametricSpectralHAR(nn.Module):
    def __init__(self, seq_len=128, in_channels=9, num_classes=6,
                 num_basis=64, hidden_dim=128, num_heads=4, dropout=0.1):
        super().__init__()

        # Enhanced spectral operator
        self.spectral_operator = EnhancedSpectralOperator(seq_len, in_channels, num_basis)

        # Spectral processing
        self.spectral_conv = nn.Sequential(
            nn.Linear(num_basis, num_basis * 2),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(num_basis * 2, num_basis)
        )
        self.spectral_attn = SpectralAttention(num_basis, num_heads, dropout)
        self.spectral_proj = nn.Linear(num_basis, hidden_dim)

        # Enhanced temporal encoder
        self.temporal_encoder = EnhancedTemporalEncoder(in_channels, hidden_dim, num_heads, dropout)

        # Enhanced fusion
        self.fusion = EnhancedCrossDomainFusion(hidden_dim, num_heads, dropout)

        # Classifier with stronger capacity
        self.classifier = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim * 2),
            nn.LayerNorm(hidden_dim * 2),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim * 2, hidden_dim),
            nn.LayerNorm(hidden_dim),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, num_classes)
        )

    def forward(self, x):
        B, C, T = x.shape

        # Spectral path
        spectral = self.spectral_operator(x)  # (B, C, F)
        spectral = self.spectral_conv(spectral)
        spectral = self.spectral_attn(spectral)

        spectral_pooled = spectral.mean(dim=1)  # (B, F)
        spectral_emb = self.spectral_proj(spectral_pooled).unsqueeze(1)  # (B, 1, D)

        # Temporal path
        temporal = self.temporal_encoder(x)  # (B, T, D)

        # Fusion
        spectral_expanded = spectral_emb.expand(B, temporal.size(1), -1)
        fused = self.fusion(temporal, spectral_expanded)

        # Classification
        logits = self.classifier(fused)
        return logits

# ============================================================================
# 6. Training Utilities with Label Smoothing & Mixup
# ============================================================================

class LabelSmoothingCrossEntropy(nn.Module):
    def __init__(self, smoothing=0.1):
        super().__init__()
        self.smoothing = smoothing

    def forward(self, pred, target):
        n_class = pred.size(1)
        one_hot = torch.zeros_like(pred).scatter(1, target.unsqueeze(1), 1)
        one_hot = one_hot * (1 - self.smoothing) + self.smoothing / n_class
        log_prob = F.log_softmax(pred, dim=1)
        loss = -(one_hot * log_prob).sum(dim=1).mean()
        return loss

def mixup_data(x, y, alpha=0.2):
    """Mixup augmentation"""
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1

    batch_size = x.size(0)
    index = torch.randperm(batch_size).to(x.device)

    mixed_x = lam * x + (1 - lam) * x[index]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam

def train_epoch(model, loader, optimizer, device, use_mixup=True):
    model.train()
    total_loss = 0
    all_preds, all_labels = [], []
    criterion = LabelSmoothingCrossEntropy(smoothing=0.1)
    criterion_ce = nn.CrossEntropyLoss()

    for x, y in loader:
        x, y = x.to(device), y.to(device)

        optimizer.zero_grad()

        if use_mixup and np.random.rand() < 0.5:
            mixed_x, y_a, y_b, lam = mixup_data(x, y, alpha=0.2)
            logits = model(mixed_x)
            loss = lam * criterion_ce(logits, y_a) + (1 - lam) * criterion_ce(logits, y_b)
        else:
            logits = model(x)
            loss = criterion(logits, y)

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()

        total_loss += loss.item()
        preds = logits.argmax(dim=1).cpu().numpy()
        all_preds.extend(preds)
        all_labels.extend(y.cpu().numpy())

    return total_loss / len(loader), accuracy_score(all_labels, all_preds)

def evaluate(model, loader, device):
    model.eval()
    all_preds, all_labels = [], []

    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            logits = model(x)
            preds = logits.argmax(dim=1).cpu().numpy()
            all_preds.extend(preds)
            all_labels.extend(y.cpu().numpy())

    return accuracy_score(all_labels, all_preds), f1_score(all_labels, all_preds, average='macro')

# ============================================================================
# 7. Main Execution
# ============================================================================

def main():
    # Config
    DATA_DIR = '/content/drive/MyDrive/Colab Notebooks/UCI-HAR/UCI-HAR'
    DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    EPOCHS = 150
    BATCH_SIZE = 64
    LR = 5e-4
    WARMUP_EPOCHS = 10

    print(f"Device: {DEVICE}")

    # Data
    train_dataset = UCIHARDataset(DATA_DIR, split='train')
    test_dataset = UCIHARDataset(DATA_DIR, split='test')

    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

    # Model
    model = EnhancedParametricSpectralHAR(
        seq_len=128, in_channels=9, num_classes=6,
        num_basis=64, hidden_dim=128, num_heads=4, dropout=0.2
    ).to(DEVICE)

    print(f"Model Parameters: {sum(p.numel() for p in model.parameters()):,}")

    # Optimizer with weight decay
    optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=5e-4)

    # Warmup + Cosine scheduler
    warmup_scheduler = torch.optim.lr_scheduler.LinearLR(
        optimizer, start_factor=0.1, end_factor=1.0, total_iters=WARMUP_EPOCHS
    )
    cosine_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
        optimizer, T_max=EPOCHS - WARMUP_EPOCHS, eta_min=1e-6
    )
    scheduler = torch.optim.lr_scheduler.SequentialLR(
        optimizer, schedulers=[warmup_scheduler, cosine_scheduler], milestones=[WARMUP_EPOCHS]
    )

    # Training
    best_acc = 0.0
    patience = 0
    MAX_PATIENCE = 30

    print("\n🚀 Starting Enhanced Training...")
    for epoch in range(EPOCHS):
        train_loss, train_acc = train_epoch(model, train_loader, optimizer, DEVICE, use_mixup=True)
        test_acc, test_f1 = evaluate(model, test_loader, DEVICE)

        scheduler.step()

        if test_acc > best_acc:
            best_acc = test_acc
            patience = 0
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'test_acc': test_acc,
                'test_f1': test_f1
            }, 'best_enhanced_har.pth')
        else:
            patience += 1

        if (epoch + 1) % 5 == 0:
            print(f"Epoch {epoch+1:03d}/{EPOCHS} | Loss: {train_loss:.4f} | "
                  f"Train: {train_acc:.4f} | Test: {test_acc:.4f} | F1: {test_f1:.4f} | "
                  f"LR: {optimizer.param_groups[0]['lr']:.6f}")

        if patience >= MAX_PATIENCE:
            print(f"Early stopping at epoch {epoch+1}")
            break

    print(f"\n✅ Best Test Accuracy: {best_acc:.4f}")

    # Interpretability
    print("\n" + "="*60)
    print("INTERPRETABILITY ANALYSIS")
    print("="*60)

    checkpoint = torch.load('best_enhanced_har.pth')
    model.load_state_dict(checkpoint['model_state_dict'])

    learned_freqs = model.spectral_operator.frequencies.detach().cpu().numpy()
    learned_freqs_hz = np.sort(learned_freqs * 50)  # Convert to Hz (50Hz sampling)

    print("Top 10 Learned Frequencies (Hz):")
    print(learned_freqs_hz[:10])
    print(f"\nDistribution: Mean={np.mean(learned_freqs_hz):.2f}Hz, Std={np.std(learned_freqs_hz):.2f}Hz")
    print(f"Final Test Acc: {checkpoint['test_acc']:.4f}, F1: {checkpoint['test_f1']:.4f}")

if __name__ == '__main__':
    main()

Device: cuda
[train] Loaded 7352 samples, shape: (7352, 9, 128)
[test] Loaded 2947 samples, shape: (2947, 9, 128)
Model Parameters: 673,369

🚀 Starting Enhanced Training...
Epoch 005/150 | Loss: 0.5087 | Train: 0.7144 | Test: 0.9223 | F1: 0.9229 | LR: 0.000275
Epoch 010/150 | Loss: 0.4811 | Train: 0.7624 | Test: 0.9169 | F1: 0.9171 | LR: 0.000500


/usr/local/lib/python3.12/dist-packages/torch/optim/lr_scheduler.py:209: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)


Epoch 015/150 | Loss: 0.4488 | Train: 0.7727 | Test: 0.9308 | F1: 0.9303 | LR: 0.000498
Epoch 020/150 | Loss: 0.4482 | Train: 0.7934 | Test: 0.9484 | F1: 0.9486 | LR: 0.000494
Epoch 025/150 | Loss: 0.4268 | Train: 0.7675 | Test: 0.9511 | F1: 0.9513 | LR: 0.000486
Epoch 030/150 | Loss: 0.4198 | Train: 0.7690 | Test: 0.9410 | F1: 0.9407 | LR: 0.000475
Epoch 035/150 | Loss: 0.3827 | Train: 0.7756 | Test: 0.9399 | F1: 0.9394 | LR: 0.000462
Epoch 040/150 | Loss: 0.4010 | Train: 0.7807 | Test: 0.9328 | F1: 0.9321 | LR: 0.000446
Epoch 045/150 | Loss: 0.4250 | Train: 0.7956 | Test: 0.9403 | F1: 0.9400 | LR: 0.000427
Epoch 050/150 | Loss: 0.3945 | Train: 0.8268 | Test: 0.9440 | F1: 0.9437 | LR: 0.000406
Epoch 055/150 | Loss: 0.4239 | Train: 0.8313 | Test: 0.9396 | F1: 0.9389 | LR: 0.000383
Epoch 060/150 | Loss: 0.3818 | Train: 0.7976 | Test: 0.9522 | F1: 0.9518 | LR: 0.000359
Epoch 065/150 | Loss: 0.3720 | Train: 0.7560 | Test: 0.9471 | F1: 0.9469 | LR: 0.000333
Epoch 070/150 | Loss: 0.3756 | T